# API Featching for CQC rating in Dudley Care Homes

In [1]:
# Import the pandas library
import requests
import json
import pandas as pd
import numpy as np
from api_key import subscription_key

In [2]:
# Base URL for the CQC API
base_url = "https://api.service.cqc.org.uk/public/v1/locations"

# Parameters for the API request
params = {
    "localAuthority": "Dudley",  
    "careHome": "Y",            
    "perPage": 100               
}

# Headers for the API request
headers = {
    # User-Agent header to avoid getting blocked
    "User-Agent": "Mozilla/5.0",  
    # Subscription key for authentication
    "Ocp-Apim-Subscription-Key": subscription_key  
}

def fetch_data(url, params, headers):
    """
    Fetch data from the API.
    """
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        # Return the JSON response if successful
        return response.json()  
    return None

def fetch_care_home_ratings(location_id, headers):
    """
    Fetch overall ratings for a care home by its location ID.
    """
    url = f"https://api.service.cqc.org.uk/public/v1/locations/{location_id}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        rating = data.get('currentRatings', {}).get('overall', {}).get('rating')
        return rating
    return None

# Fetch data from the CQC API
data = fetch_data(base_url, params, headers)

In [3]:
# Check if data was successfully fetched
if data:
    # Process and print the data as needed
    for location in data.get('locations', []):
        name = location.get('locationName')
        postal_code = location.get('postalCode')
        rating = fetch_care_home_ratings(location.get('locationId'), headers)

        # Construct the full address
        full_address = f"{postal_code}".replace(" ,", "").replace(" ,", "").replace(" ,", "").strip(", ")

        print(f"Care Home: {name}")
        print(f"PostCode: {postal_code}")
        print(f"Overall Rating: {rating}")
        print()
else:
    print("Data fetch was unsuccessful.")


Care Home: Cygnet House
PostCode: DY8 3UU
Overall Rating: Good

Care Home: Parkfield Grange
PostCode: DY8 1QH
Overall Rating: None

Care Home: 60 Olive Lane
PostCode: B62 8LZ
Overall Rating: Good

Care Home: Arcare for Forte Limited
PostCode: WV14 9QB
Overall Rating: Good

Care Home: Hollybush House Nursing Home
PostCode: DY8 4HZ
Overall Rating: Good

Care Home: Woodview House Nursing Home
PostCode: B63 1EN
Overall Rating: Requires improvement

Care Home: Commonside Care Limited - 73 Commonside
PostCode: DY5 4AJ
Overall Rating: Good

Care Home: Stanford House
PostCode: DY3 1ST
Overall Rating: Requires improvement

Care Home: Tudor Manor
PostCode: DY8 3XF
Overall Rating: Inadequate

Care Home: Attwood Street
PostCode: B63 3UE
Overall Rating: Good

Care Home: Langstone Road
PostCode: DY1 2NJ
Overall Rating: No published rating

Care Home: Stickley Lane
PostCode: DY3 2JQ
Overall Rating: Requires improvement

Care Home: Edmore House Residential Home
PostCode: DY2 7TB
Overall Rating: Good



In [5]:
# Create a list of dictionaries containing care home data using list comprehension
data_list = [
    {
        # Extract the care home name
        'Care Home': location.get('locationName'),  
        # Extract the postal code
        'Postal Code': location.get('postalCode'),
        # Fetch the care home rating
        'Rating': fetch_care_home_ratings(location.get('locationId'), headers) 
    }
    
    # Iterate over each location in the data
    for location in data.get('locations', [])
]

# Convert the list of dictionaries to a DataFrame
ratings_df = pd.DataFrame(data_list)

# Print the first 50 entries of the DataFrame
print(ratings_df.head(50))


                                      Care Home Postal Code  \
0                                  Cygnet House     DY8 3UU   
1                              Parkfield Grange     DY8 1QH   
2                                 60 Olive Lane     B62 8LZ   
3                      Arcare for Forte Limited    WV14 9QB   
4                  Hollybush House Nursing Home     DY8 4HZ   
5                   Woodview House Nursing Home     B63 1EN   
6       Commonside Care Limited - 73 Commonside     DY5 4AJ   
7                                Stanford House     DY3 1ST   
8                                   Tudor Manor     DY8 3XF   
9                                Attwood Street     B63 3UE   
10                               Langstone Road     DY1 2NJ   
11                                Stickley Lane     DY3 2JQ   
12                Edmore House Residential Home     DY2 7TB   
13              Belvidere Residential Care Home     DY1 2DH   
14                     Woodlands Gate Rest Home     DY9

In [6]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Care Home    100 non-null    object
 1   Postal Code  100 non-null    object
 2   Rating       90 non-null     object
dtypes: object(3)
memory usage: 2.5+ KB


## Merging the Nursing PDF and API CQC Rating

In [7]:
# Load the CSV file
file_path = "../data/nhomes_cleaned_coordinates.csv"
nhomes_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to understand its structure
nhomes_df.head()

,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \r\nNursing,Capacity,Post Code,Care Home Name,Min Age,Max Age,Coordinates,Latitude,Longitude
0,Abbeygate Care Centre 2 Leys Road Brockmoor Br...,abbeygatecare1@gmail.com,01384 571295,Age 65+,Dementia \r\nMental Health Condition \r\nOld A...,No,17.0,DY5 3UR,Abbeygate Care,65.0,NaN,"[52.4857514, -2.1341733]",52.485751,-2.134173
1,Abbeymere 12 Eggington Road Wollaston Stourbri...,abbeymere@karelink.co.uk,01384 395195,Ages 65+,Dementia \r\nMental Health Condition \r\nOld A...,No,18.0,DY8 2QJ,Abbeymere,65.0,NaN,"[52.4584, -2.16295]",52.458400,-2.162950
2,Allenbrook Nursing Home 209 Spies Lane Halesow...,manager@allenbrooknursing\r\nhome.co.uk,0121 422 5844,Ages 55+,Dementia \r\nMental Health Condition \r\nNo Me...,Yes,36.0,B62 9SJ,Allenbrook Nursing,55.0,NaN,"[52.4613003, -2.0139452]",52.461300,-2.013945
3,Amberley Care Home 481-483 Stourbridge Road Ha...,amberleycarehome@hotmail\r\n.co.uk,01384 482365,Ages 65+,Dementia \r\nOld Age,No,25.0,DY5 1LB,Amberley Care Home,65.0,NaN,"[52.4927718, -2.1164204]",52.492772,-2.116420
4,Arcare For Forte 440 Birmingham New Road Bilst...,ksharma@arcarehomes.co.u\r\nk,01902 880108,18+,Learning Disability \r\nMental Health Conditio...,No,9.0,WV14 9QB,Arcare For Forte,18.0,NaN,"[52.5481562, -2.094362]",52.548156,-2.094362


In [16]:
# Ensure postal code columns in both DataFrames are properly formatted
nhomes_df['Post Code'] = nhomes_df['Post Code'].str.strip()
ratings_df['Postal Code'] = ratings_df['Postal Code'].str.strip()

# Filter nhomes_df for postal codes starting with 'DY' and merge with ratings
filtered_nh_df = nhomes_df[nhomes_df['Post Code'].str.startswith('DY', na=False)].copy()
filtered_nh_df['Post Code'] = filtered_nh_df['Post Code'].str.strip()

# Merge the filtered DataFrame with ratings
merged_df = pd.merge(filtered_nh_df, ratings_df, left_on='Post Code', right_on='Postal Code', how='inner')

# To include ratings in the original nhomes_df
final_df = pd.merge(nhomes_df, merged_df[['Post Code', 'Rating']], on='Post Code', how='left')

# Display the merged DataFrame
final_df.tail(5)

,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \r\nNursing,Capacity,Post Code,Care Home Name,Min Age,Max Age,Coordinates,Latitude,Longitude,Rating
113,Woodview House Nursing Home Waugh Drive Hagley...,woodview@shaw.co.uk,0121 585 6440,Ages 65+,C\r\nMental Health Condition \r\nOld Age \r\nC...,Yes,24.0,B63 1EN,Woodview House,65.0,NaN,"[52.4356855, -2.0820804]",52.435685,-2.082080,NaN
114,Wordsley Hall Mill Street Wordsley Stourbridge...,manager.wordsleyhall@min\r\nstercaregroup.co.uk,01384 571606,Ages 65+,"Old Age, Dementia",No,41.0,DY8 5SX,Wordsley Hall,65.0,NaN,"[52.4780241, -2.1562237]",52.478024,-2.156224,Good
115,Wychbury Care Home 350-352 Hagley Road Pedmore...,daniel@wychcare.com,01562 885106,Ages 65+,", Old Age",No,42.0,DY9 0QY,Wychbury Care Home,65.0,NaN,"[52.4327421, -2.1301692]",52.432742,-2.130169,Good
116,Wyncroft House 16 Moss Grove Kingswinford DY6 9HU,manager@wyncrofthouse.c\r\no.uk,01384 291688,Ages 65+,"Old Age, Dementia",Yes,38.0,DY6 9HU,Wyncroft House,65.0,NaN,"[52.5025546, -2.1702452]",52.502555,-2.170245,Requires improvement
117,Wyncroft House 16 Moss Grove Kingswinford DY6 9HU,manager@wyncrofthouse.c\r\no.uk,01384 291688,Ages 65+,"Old Age, Dementia",Yes,38.0,DY6 9HU,Wyncroft House,65.0,NaN,"[52.5025546, -2.1702452]",52.502555,-2.170245,None


## Cleaning of the Merged DataFrame

In [17]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name & Address      118 non-null    object 
 1   Email               118 non-null    object 
 2   Telephone No        116 non-null    object 
 3   Age Range           110 non-null    object 
 4   CQC Registered For  110 non-null    object 
 5   Provide 
Nursing   110 non-null    object 
 6   Capacity            110 non-null    float64
 7   Post Code           118 non-null    object 
 8   Care Home Name      118 non-null    object 
 9   Min Age             97 non-null     float64
 10  Max Age             10 non-null     float64
 11  Coordinates         118 non-null    object 
 12  Latitude            118 non-null    float64
 13  Longitude           118 non-null    float64
 14  Rating              86 non-null     object 
dtypes: float64(5), object(10)
memory usage: 14.0+ KB


In [18]:
# Remove the 'Min Age' and 'Max Age' columns
cleaned_df = final_df.drop(['Min Age', 'Max Age'], axis=1)

# Verify the columns of the cleaned DataFrame
print(cleaned_df.columns)

Index(['Name & Address', 'Email', 'Telephone No', 'Age Range',
       'CQC Registered For', 'Provide \r\nNursing', 'Capacity', 'Post Code',
       'Care Home Name', 'Coordinates', 'Latitude', 'Longitude', 'Rating'],
      dtype='object')


In [19]:
# Check for duplicate rows based on the 'Name & Address' column
duplicates = cleaned_df.duplicated(subset=['Name & Address'])

# Print the duplicate rows, if any
print(cleaned_df[duplicates])

                                        Name & Address  \
7    Ashgrove Nursing Home 9 Dudley Wood Road Nethe...   
14   Camelot Rest Home 152 Stourbridge Road Holy Ha...   
15   Camelot Rest Home 152 Stourbridge Road Holy Ha...   
16   Camelot Rest Home 152 Stourbridge Road Holy Ha...   
17   Camelot Rest Home 152 Stourbridge Road Holy Ha...   
18   Camelot Rest Home 152 Stourbridge Road Holy Ha...   
19   Camelot Rest Home 152 Stourbridge Road Holy Ha...   
20   Camelot Rest Home 152 Stourbridge Road Holy Ha...   
22            Castle View 6 Priory Road Dudley DY1 4AD   
24   Castlewellan House 41 Moss Grove Kingswinford ...   
38             Highview 12 Priory Road Dudlley DY1 4AD   
42      Holly Hall 170 Stourbridge Road Dudley DY1 2ER   
43      Holly Hall 170 Stourbridge Road Dudley DY1 2ER   
44      Holly Hall 170 Stourbridge Road Dudley DY1 2ER   
45      Holly Hall 170 Stourbridge Road Dudley DY1 2ER   
46      Holly Hall 170 Stourbridge Road Dudley DY1 2ER   
47      Holly 

In [22]:
# Define a custom order for the ratings
rating_priority = {'Outstanding': 1, 'Good': 2, 'Requires improvement': 3, 'Inadequate': 4, 'No published rating': 5}

# Add a column for the custom rating priority
cleaned_df['RatingPriority'] = cleaned_df['Rating'].map(rating_priority)

# Sort the DataFrame by Name & Address, Email, Telephone No, Post Code, and RatingPriority
df_sorted = cleaned_df.sort_values(by=['Name & Address', 'Email', 'Telephone No', 'Post Code', 'RatingPriority'])

# Drop duplicates, keeping the first (which will have the highest priority due to sorting)
selected_columns = ['Name & Address', 'Email', 'Telephone No', 'Post Code']
df_unique = df_sorted.drop_duplicates(subset=selected_columns, keep='first')

# Optionally, remove the 'RatingPriority' column as it's no longer needed
df_unique = df_unique.drop(columns=['RatingPriority'])

# keep track of removed duplicates
removed_duplicates = df_sorted[~df_sorted.index.isin(df_unique.index)]

# Now, df_unique contains the DataFrame with duplicates removed, prioritized by the custom rating order
df_unique

,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \r\nNursing,Capacity,Post Code,Care Home Name,Coordinates,Latitude,Longitude,Rating
25,"48 Cedar Road (Voyage) Dudley, DY1 4HW",48CedarRoad@voyagecare.\r\ncom,01384 241877,Under 65,Learning Disability \r\nMental Health Conditio...,No,9.0,DY1 4HW,48 Cedar Road,"[52.5, -2.08333]",52.500000,-2.083330,No published rating
0,Abbeygate Care Centre 2 Leys Road Brockmoor Br...,abbeygatecare1@gmail.com,01384 571295,Age 65+,Dementia \r\nMental Health Condition \r\nOld A...,No,17.0,DY5 3UR,Abbeygate Care,"[52.4857514, -2.1341733]",52.485751,-2.134173,No published rating
1,Abbeymere 12 Eggington Road Wollaston Stourbri...,abbeymere@karelink.co.uk,01384 395195,Ages 65+,Dementia \r\nMental Health Condition \r\nOld A...,No,18.0,DY8 2QJ,Abbeymere,"[52.4584, -2.16295]",52.458400,-2.162950,NaN
2,Allenbrook Nursing Home 209 Spies Lane Halesow...,manager@allenbrooknursing\r\nhome.co.uk,0121 422 5844,Ages 55+,Dementia \r\nMental Health Condition \r\nNo Me...,Yes,36.0,B62 9SJ,Allenbrook Nursing,"[52.4613003, -2.0139452]",52.461300,-2.013945,NaN
3,Amberley Care Home 481-483 Stourbridge Road Ha...,amberleycarehome@hotmail\r\n.co.uk,01384 482365,Ages 65+,Dementia \r\nOld Age,No,25.0,DY5 1LB,Amberley Care Home,"[52.4927718, -2.1164204]",52.492772,-2.116420,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Woodlands Gate Rest Home 12 Dingle Road Pedmor...,woodlands@sclcare.co.uk,01562 885546,Ages 60+,Old Age \r\nPhysical Disability \r\nSensory Im...,No,20.0,DY9 0RS,Woodlands Gate Rest,"[52.4434178, -2.1320989]",52.443418,-2.132099,Good
113,Woodview House Nursing Home Waugh Drive Hagley...,woodview@shaw.co.uk,0121 585 6440,Ages 65+,C\r\nMental Health Condition \r\nOld Age \r\nC...,Yes,24.0,B63 1EN,Woodview House,"[52.4356855, -2.0820804]",52.435685,-2.082080,NaN
114,Wordsley Hall Mill Street Wordsley Stourbridge...,manager.wordsleyhall@min\r\nstercaregroup.co.uk,01384 571606,Ages 65+,"Old Age, Dementia",No,41.0,DY8 5SX,Wordsley Hall,"[52.4780241, -2.1562237]",52.478024,-2.156224,Good
115,Wychbury Care Home 350-352 Hagley Road Pedmore...,daniel@wychcare.com,01562 885106,Ages 65+,", Old Age",No,42.0,DY9 0QY,Wychbury Care Home,"[52.4327421, -2.1301692]",52.432742,-2.130169,Good


In [25]:
removed_duplicates

,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \r\nNursing,Capacity,Post Code,Care Home Name,Coordinates,Latitude,Longitude,Rating,RatingPriority
6,Ashgrove Nursing Home 9 Dudley Wood Road Nethe...,cea@ashgrovecare.com,01384 413913,Ages 65+,Dementia \r\nOld Age \r\nSensory Impairment,Yes,57.0,DY2 0DA,Ashgrove Nursing,"[52.4777428, -2.0859231]",52.477743,-2.085923,No published rating,5.0
15,Camelot Rest Home 152 Stourbridge Road Holy Ha...,cam152@hotmail.co.uk,01384 214290,Ages 18+,Dementia \r\nOld Age,No,33.0,DY1 2ER,Camelot Rest Home,"[52.5028747, -2.1047859]",52.502875,-2.104786,Requires improvement,3.0
17,Camelot Rest Home 152 Stourbridge Road Holy Ha...,cam152@hotmail.co.uk,01384 214290,Ages 18+,Dementia \r\nOld Age,No,33.0,DY1 2ER,Camelot Rest Home,"[52.5028747, -2.1047859]",52.502875,-2.104786,Requires improvement,3.0
19,Camelot Rest Home 152 Stourbridge Road Holy Ha...,cam152@hotmail.co.uk,01384 214290,Ages 18+,Dementia \r\nOld Age,No,33.0,DY1 2ER,Camelot Rest Home,"[52.5028747, -2.1047859]",52.502875,-2.104786,Requires improvement,3.0
14,Camelot Rest Home 152 Stourbridge Road Holy Ha...,cam152@hotmail.co.uk,01384 214290,Ages 18+,Dementia \r\nOld Age,No,33.0,DY1 2ER,Camelot Rest Home,"[52.5028747, -2.1047859]",52.502875,-2.104786,No published rating,5.0
16,Camelot Rest Home 152 Stourbridge Road Holy Ha...,cam152@hotmail.co.uk,01384 214290,Ages 18+,Dementia \r\nOld Age,No,33.0,DY1 2ER,Camelot Rest Home,"[52.5028747, -2.1047859]",52.502875,-2.104786,No published rating,5.0
18,Camelot Rest Home 152 Stourbridge Road Holy Ha...,cam152@hotmail.co.uk,01384 214290,Ages 18+,Dementia \r\nOld Age,No,33.0,DY1 2ER,Camelot Rest Home,"[52.5028747, -2.1047859]",52.502875,-2.104786,No published rating,5.0
20,Camelot Rest Home 152 Stourbridge Road Holy Ha...,cam152@hotmail.co.uk,01384 214290,Ages 18+,Dementia \r\nOld Age,No,33.0,DY1 2ER,Camelot Rest Home,"[52.5028747, -2.1047859]",52.502875,-2.104786,No published rating,5.0
22,Castle View 6 Priory Road Dudley DY1 4AD,goldenseniorcare@outlook.\r\ncom,01384 253426,Ages 18+,Dementia \r\nOld Age,No,13.0,DY1 4AD,Castle View,"[52.513868, -2.0858346]",52.513868,-2.085835,Good,2.0
24,Castlewellan House 41 Moss Grove Kingswinford ...,admin@horizontc.com,01384 298321,Ages 65+,Old Age,No,18.0,DY6 9HP,Castlewellan House,"[52.5015863, -2.1698679]",52.501586,-2.169868,Requires improvement,3.0


In [26]:
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74 entries, 25 to 116
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name & Address      74 non-null     object 
 1   Email               74 non-null     object 
 2   Telephone No        72 non-null     object 
 3   Age Range           73 non-null     object 
 4   CQC Registered For  73 non-null     object 
 5   Provide 
Nursing   73 non-null     object 
 6   Capacity            73 non-null     float64
 7   Post Code           74 non-null     object 
 8   Care Home Name      74 non-null     object 
 9   Coordinates         74 non-null     object 
 10  Latitude            74 non-null     float64
 11  Longitude           74 non-null     float64
 12  Rating              74 non-null     object 
dtypes: float64(3), object(10)
memory usage: 8.1+ KB


In [27]:
# Reset the index
df_unique.reset_index(drop=True, inplace=True)

df_unique.head(74)

,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \r\nNursing,Capacity,Post Code,Care Home Name,Coordinates,Latitude,Longitude,Rating
0,"48 Cedar Road (Voyage) Dudley, DY1 4HW",48CedarRoad@voyagecare.\r\ncom,01384 241877,Under 65,Learning Disability \r\nMental Health Conditio...,No,9.0,DY1 4HW,48 Cedar Road,"[52.5, -2.08333]",52.500000,-2.083330,No published rating
1,Abbeygate Care Centre 2 Leys Road Brockmoor Br...,abbeygatecare1@gmail.com,01384 571295,Age 65+,Dementia \r\nMental Health Condition \r\nOld A...,No,17.0,DY5 3UR,Abbeygate Care,"[52.4857514, -2.1341733]",52.485751,-2.134173,No published rating
2,Abbeymere 12 Eggington Road Wollaston Stourbri...,abbeymere@karelink.co.uk,01384 395195,Ages 65+,Dementia \r\nMental Health Condition \r\nOld A...,No,18.0,DY8 2QJ,Abbeymere,"[52.4584, -2.16295]",52.458400,-2.162950,No published rating
3,Allenbrook Nursing Home 209 Spies Lane Halesow...,manager@allenbrooknursing\r\nhome.co.uk,0121 422 5844,Ages 55+,Dementia \r\nMental Health Condition \r\nNo Me...,Yes,36.0,B62 9SJ,Allenbrook Nursing,"[52.4613003, -2.0139452]",52.461300,-2.013945,No published rating
4,Amberley Care Home 481-483 Stourbridge Road Ha...,amberleycarehome@hotmail\r\n.co.uk,01384 482365,Ages 65+,Dementia \r\nOld Age,No,25.0,DY5 1LB,Amberley Care Home,"[52.4927718, -2.1164204]",52.492772,-2.116420,No published rating
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Woodlands Gate Rest Home 12 Dingle Road Pedmor...,woodlands@sclcare.co.uk,01562 885546,Ages 60+,Old Age \r\nPhysical Disability \r\nSensory Im...,No,20.0,DY9 0RS,Woodlands Gate Rest,"[52.4434178, -2.1320989]",52.443418,-2.132099,Good
70,Woodview House Nursing Home Waugh Drive Hagley...,woodview@shaw.co.uk,0121 585 6440,Ages 65+,C\r\nMental Health Condition \r\nOld Age \r\nC...,Yes,24.0,B63 1EN,Woodview House,"[52.4356855, -2.0820804]",52.435685,-2.082080,No published rating
71,Wordsley Hall Mill Street Wordsley Stourbridge...,manager.wordsleyhall@min\r\nstercaregroup.co.uk,01384 571606,Ages 65+,"Old Age, Dementia",No,41.0,DY8 5SX,Wordsley Hall,"[52.4780241, -2.1562237]",52.478024,-2.156224,Good
72,Wychbury Care Home 350-352 Hagley Road Pedmore...,daniel@wychcare.com,01562 885106,Ages 65+,", Old Age",No,42.0,DY9 0QY,Wychbury Care Home,"[52.4327421, -2.1301692]",52.432742,-2.130169,Good


In [28]:
# Replace None and NaN values with 'No published rating'
df_unique.loc[df_unique['Rating'].isnull(), 'Rating'] = 'No published rating'

# Display the first 74 rows
df_unique.head(74)

,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \r\nNursing,Capacity,Post Code,Care Home Name,Coordinates,Latitude,Longitude,Rating
0,"48 Cedar Road (Voyage) Dudley, DY1 4HW",48CedarRoad@voyagecare.\r\ncom,01384 241877,Under 65,Learning Disability \r\nMental Health Conditio...,No,9.0,DY1 4HW,48 Cedar Road,"[52.5, -2.08333]",52.500000,-2.083330,No published rating
1,Abbeygate Care Centre 2 Leys Road Brockmoor Br...,abbeygatecare1@gmail.com,01384 571295,Age 65+,Dementia \r\nMental Health Condition \r\nOld A...,No,17.0,DY5 3UR,Abbeygate Care,"[52.4857514, -2.1341733]",52.485751,-2.134173,No published rating
2,Abbeymere 12 Eggington Road Wollaston Stourbri...,abbeymere@karelink.co.uk,01384 395195,Ages 65+,Dementia \r\nMental Health Condition \r\nOld A...,No,18.0,DY8 2QJ,Abbeymere,"[52.4584, -2.16295]",52.458400,-2.162950,No published rating
3,Allenbrook Nursing Home 209 Spies Lane Halesow...,manager@allenbrooknursing\r\nhome.co.uk,0121 422 5844,Ages 55+,Dementia \r\nMental Health Condition \r\nNo Me...,Yes,36.0,B62 9SJ,Allenbrook Nursing,"[52.4613003, -2.0139452]",52.461300,-2.013945,No published rating
4,Amberley Care Home 481-483 Stourbridge Road Ha...,amberleycarehome@hotmail\r\n.co.uk,01384 482365,Ages 65+,Dementia \r\nOld Age,No,25.0,DY5 1LB,Amberley Care Home,"[52.4927718, -2.1164204]",52.492772,-2.116420,No published rating
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Woodlands Gate Rest Home 12 Dingle Road Pedmor...,woodlands@sclcare.co.uk,01562 885546,Ages 60+,Old Age \r\nPhysical Disability \r\nSensory Im...,No,20.0,DY9 0RS,Woodlands Gate Rest,"[52.4434178, -2.1320989]",52.443418,-2.132099,Good
70,Woodview House Nursing Home Waugh Drive Hagley...,woodview@shaw.co.uk,0121 585 6440,Ages 65+,C\r\nMental Health Condition \r\nOld Age \r\nC...,Yes,24.0,B63 1EN,Woodview House,"[52.4356855, -2.0820804]",52.435685,-2.082080,No published rating
71,Wordsley Hall Mill Street Wordsley Stourbridge...,manager.wordsleyhall@min\r\nstercaregroup.co.uk,01384 571606,Ages 65+,"Old Age, Dementia",No,41.0,DY8 5SX,Wordsley Hall,"[52.4780241, -2.1562237]",52.478024,-2.156224,Good
72,Wychbury Care Home 350-352 Hagley Road Pedmore...,daniel@wychcare.com,01562 885106,Ages 65+,", Old Age",No,42.0,DY9 0QY,Wychbury Care Home,"[52.4327421, -2.1301692]",52.432742,-2.130169,Good


In [30]:
# Replace NaN values in 'Capacity' column with a default value (e.g., 0)
df_unique.loc[:, 'Capacity'] = df_unique['Capacity'].fillna(0)

# Convert 'Capacity' column to integers
df_unique.loc[:, 'Capacity'] = df_unique['Capacity'].astype(int)

# Display the DataFrame after conversion
df_unique.head()

,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \r\nNursing,Capacity,Post Code,Care Home Name,Coordinates,Latitude,Longitude,Rating
0,"48 Cedar Road (Voyage) Dudley, DY1 4HW",48CedarRoad@voyagecare.\r\ncom,01384 241877,Under 65,Learning Disability \r\nMental Health Conditio...,No,9.0,DY1 4HW,48 Cedar Road,"[52.5, -2.08333]",52.500000,-2.083330,No published rating
1,Abbeygate Care Centre 2 Leys Road Brockmoor Br...,abbeygatecare1@gmail.com,01384 571295,Age 65+,Dementia \r\nMental Health Condition \r\nOld A...,No,17.0,DY5 3UR,Abbeygate Care,"[52.4857514, -2.1341733]",52.485751,-2.134173,No published rating
2,Abbeymere 12 Eggington Road Wollaston Stourbri...,abbeymere@karelink.co.uk,01384 395195,Ages 65+,Dementia \r\nMental Health Condition \r\nOld A...,No,18.0,DY8 2QJ,Abbeymere,"[52.4584, -2.16295]",52.458400,-2.162950,No published rating
3,Allenbrook Nursing Home 209 Spies Lane Halesow...,manager@allenbrooknursing\r\nhome.co.uk,0121 422 5844,Ages 55+,Dementia \r\nMental Health Condition \r\nNo Me...,Yes,36.0,B62 9SJ,Allenbrook Nursing,"[52.4613003, -2.0139452]",52.461300,-2.013945,No published rating
4,Amberley Care Home 481-483 Stourbridge Road Ha...,amberleycarehome@hotmail\r\n.co.uk,01384 482365,Ages 65+,Dementia \r\nOld Age,No,25.0,DY5 1LB,Amberley Care Home,"[52.4927718, -2.1164204]",52.492772,-2.116420,No published rating


In [31]:
# Remove 'Ages' and 'Age' from the 'Age Range' column
df_unique.loc[:, 'Age Range'] = df_unique['Age Range'].str.replace('Ages', '').str.replace('Age', '').str.strip()

# Display the DataFrame after modification
df_unique.tail()

,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \r\nNursing,Capacity,Post Code,Care Home Name,Coordinates,Latitude,Longitude,Rating
69,Woodlands Gate Rest Home 12 Dingle Road Pedmor...,woodlands@sclcare.co.uk,01562 885546,60+,Old Age \r\nPhysical Disability \r\nSensory Im...,No,20.0,DY9 0RS,Woodlands Gate Rest,"[52.4434178, -2.1320989]",52.443418,-2.132099,Good
70,Woodview House Nursing Home Waugh Drive Hagley...,woodview@shaw.co.uk,0121 585 6440,65+,C\r\nMental Health Condition \r\nOld Age \r\nC...,Yes,24.0,B63 1EN,Woodview House,"[52.4356855, -2.0820804]",52.435685,-2.082080,No published rating
71,Wordsley Hall Mill Street Wordsley Stourbridge...,manager.wordsleyhall@min\r\nstercaregroup.co.uk,01384 571606,65+,"Old Age, Dementia",No,41.0,DY8 5SX,Wordsley Hall,"[52.4780241, -2.1562237]",52.478024,-2.156224,Good
72,Wychbury Care Home 350-352 Hagley Road Pedmore...,daniel@wychcare.com,01562 885106,65+,", Old Age",No,42.0,DY9 0QY,Wychbury Care Home,"[52.4327421, -2.1301692]",52.432742,-2.130169,Good
73,Wyncroft House 16 Moss Grove Kingswinford DY6 9HU,manager@wyncrofthouse.c\r\no.uk,01384 291688,65+,"Old Age, Dementia",Yes,38.0,DY6 9HU,Wyncroft House,"[52.5025546, -2.1702452]",52.502555,-2.170245,Requires improvement


In [33]:
# Get a list of columns in the DataFrame
columns = df_unique.columns.tolist()

# Move "Care Home Name" to the front of the list
columns.insert(0, columns.pop(columns.index("Care Home Name")))

# Reorder the columns in the DataFrame
df_unique = df_unique[columns]

# Display DateFrame 
df_unique

,Care Home Name,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \r\nNursing,Capacity,Post Code,Coordinates,Latitude,Longitude,Rating
0,48 Cedar Road,"48 Cedar Road (Voyage) Dudley, DY1 4HW",48CedarRoad@voyagecare.\r\ncom,01384 241877,Under 65,Learning Disability \r\nMental Health Conditio...,No,9.0,DY1 4HW,"[52.5, -2.08333]",52.500000,-2.083330,No published rating
1,Abbeygate Care,Abbeygate Care Centre 2 Leys Road Brockmoor Br...,abbeygatecare1@gmail.com,01384 571295,65+,Dementia \r\nMental Health Condition \r\nOld A...,No,17.0,DY5 3UR,"[52.4857514, -2.1341733]",52.485751,-2.134173,No published rating
2,Abbeymere,Abbeymere 12 Eggington Road Wollaston Stourbri...,abbeymere@karelink.co.uk,01384 395195,65+,Dementia \r\nMental Health Condition \r\nOld A...,No,18.0,DY8 2QJ,"[52.4584, -2.16295]",52.458400,-2.162950,No published rating
3,Allenbrook Nursing,Allenbrook Nursing Home 209 Spies Lane Halesow...,manager@allenbrooknursing\r\nhome.co.uk,0121 422 5844,55+,Dementia \r\nMental Health Condition \r\nNo Me...,Yes,36.0,B62 9SJ,"[52.4613003, -2.0139452]",52.461300,-2.013945,No published rating
4,Amberley Care Home,Amberley Care Home 481-483 Stourbridge Road Ha...,amberleycarehome@hotmail\r\n.co.uk,01384 482365,65+,Dementia \r\nOld Age,No,25.0,DY5 1LB,"[52.4927718, -2.1164204]",52.492772,-2.116420,No published rating
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Woodlands Gate Rest,Woodlands Gate Rest Home 12 Dingle Road Pedmor...,woodlands@sclcare.co.uk,01562 885546,60+,Old Age \r\nPhysical Disability \r\nSensory Im...,No,20.0,DY9 0RS,"[52.4434178, -2.1320989]",52.443418,-2.132099,Good
70,Woodview House,Woodview House Nursing Home Waugh Drive Hagley...,woodview@shaw.co.uk,0121 585 6440,65+,C\r\nMental Health Condition \r\nOld Age \r\nC...,Yes,24.0,B63 1EN,"[52.4356855, -2.0820804]",52.435685,-2.082080,No published rating
71,Wordsley Hall,Wordsley Hall Mill Street Wordsley Stourbridge...,manager.wordsleyhall@min\r\nstercaregroup.co.uk,01384 571606,65+,"Old Age, Dementia",No,41.0,DY8 5SX,"[52.4780241, -2.1562237]",52.478024,-2.156224,Good
72,Wychbury Care Home,Wychbury Care Home 350-352 Hagley Road Pedmore...,daniel@wychcare.com,01562 885106,65+,", Old Age",No,42.0,DY9 0QY,"[52.4327421, -2.1301692]",52.432742,-2.130169,Good


In [35]:
# Replace the first data point in the 'Care Home Name' column with 'Voyage care'
df_unique.at[0, 'Care Home Name'] = 'Voyage care'

# Print 
df_unique

,Care Home Name,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \r\nNursing,Capacity,Post Code,Coordinates,Latitude,Longitude,Rating
0,Voyage care,"48 Cedar Road (Voyage) Dudley, DY1 4HW",48CedarRoad@voyagecare.\r\ncom,01384 241877,Under 65,Learning Disability \r\nMental Health Conditio...,No,9.0,DY1 4HW,"[52.5, -2.08333]",52.500000,-2.083330,No published rating
1,Abbeygate Care,Abbeygate Care Centre 2 Leys Road Brockmoor Br...,abbeygatecare1@gmail.com,01384 571295,65+,Dementia \r\nMental Health Condition \r\nOld A...,No,17.0,DY5 3UR,"[52.4857514, -2.1341733]",52.485751,-2.134173,No published rating
2,Abbeymere,Abbeymere 12 Eggington Road Wollaston Stourbri...,abbeymere@karelink.co.uk,01384 395195,65+,Dementia \r\nMental Health Condition \r\nOld A...,No,18.0,DY8 2QJ,"[52.4584, -2.16295]",52.458400,-2.162950,No published rating
3,Allenbrook Nursing,Allenbrook Nursing Home 209 Spies Lane Halesow...,manager@allenbrooknursing\r\nhome.co.uk,0121 422 5844,55+,Dementia \r\nMental Health Condition \r\nNo Me...,Yes,36.0,B62 9SJ,"[52.4613003, -2.0139452]",52.461300,-2.013945,No published rating
4,Amberley Care Home,Amberley Care Home 481-483 Stourbridge Road Ha...,amberleycarehome@hotmail\r\n.co.uk,01384 482365,65+,Dementia \r\nOld Age,No,25.0,DY5 1LB,"[52.4927718, -2.1164204]",52.492772,-2.116420,No published rating
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Woodlands Gate Rest,Woodlands Gate Rest Home 12 Dingle Road Pedmor...,woodlands@sclcare.co.uk,01562 885546,60+,Old Age \r\nPhysical Disability \r\nSensory Im...,No,20.0,DY9 0RS,"[52.4434178, -2.1320989]",52.443418,-2.132099,Good
70,Woodview House,Woodview House Nursing Home Waugh Drive Hagley...,woodview@shaw.co.uk,0121 585 6440,65+,C\r\nMental Health Condition \r\nOld Age \r\nC...,Yes,24.0,B63 1EN,"[52.4356855, -2.0820804]",52.435685,-2.082080,No published rating
71,Wordsley Hall,Wordsley Hall Mill Street Wordsley Stourbridge...,manager.wordsleyhall@min\r\nstercaregroup.co.uk,01384 571606,65+,"Old Age, Dementia",No,41.0,DY8 5SX,"[52.4780241, -2.1562237]",52.478024,-2.156224,Good
72,Wychbury Care Home,Wychbury Care Home 350-352 Hagley Road Pedmore...,daniel@wychcare.com,01562 885106,65+,", Old Age",No,42.0,DY9 0QY,"[52.4327421, -2.1301692]",52.432742,-2.130169,Good


## Importing into a CSV file

In [38]:
output_file_path = "../data/CareHome-data.csv"

# Export the DataFrame to a CSV file
df_unique.to_csv(output_file_path, index=False)

print("Data has been successfully exported to:", output_file_path)

Data has been successfully exported to: ../data/CareHome-data.csv
